# PS106A Voter Summary Analysis - Geographic Data

Author: Ian Castro, Fall 2020

Hello! This notebook is designed to help speed up your analysis for the Political Science 106A: Campaign Media voter summary assignment. If you are unfamiliar with Jupyter notebooks, these are ways data scientists share Python code and analyses in an easy-to-read format. You do not need to know how to use Python to use this notebook, but if you are interested, check out the offerings from UC Berkeley's Data Science Education Program. Please refer to Professor Schnur's emails and the bCourses for the full prompt and all parts necessary for this assignment.

For this notebook, we will focus on the **geographic** data analysis which will take most of your time. For the demographic analysis, I recommend working on that strictly in an Excel/Google sheet and keeping your analysis there, because there is a lot less data and it should already be grouped. 

In short, to use this notebook, you need to:
> 1. Upload your data to the directory with this notebook
> 2. Run cells by pressing Shift + Enter

*Make sure you follow the instructions to ensure that this works.* You can find example datasets in bCourses.

Best of luck! If you have any questions or issues navigating this assignment, email castro.ian@berkeley.edu.

## Step 1: Geographic Data Collection

To start this assignment, please go to https://www.truckads.com/ to find the media markets and counties in your state for your particular campaign. **MAKE SURE YOU USE THE EXACT SAME COLUMN NAMES IN THE INSTRUCTIONS, OR ELSE THIS WILL NOT WORK.** 


First, open Excel or some other spreadsheet software. In that sheet, create 2 columns: `Media Market` and `County`. In each row, list the county in that state with its corresponding Media Market. For example:

| Media Market | County |
| ----------   | ------ |
| Pittsburgh   | Allegheny |


Save this file in Excel **as a csv**, with the file name as **"media_markets.csv"** and upload it to the directory of this notebook. You can do so by clicking on the toolbar the top, then File > Open... This will open a new tab. In that tab, click on Upload on the top right and upload your file to the directory.


In [ ]:
## Importing the Data 8 modules for analysis and visualization. 
# Please run this cell by clicking Run at the top
# or pressing shift + enter.
from datascience import *
import numpy as np
import matplotlib
import pandas as pd
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
# Now, run this cell by pressing shift + enter. 
# You should see a table appear if you did this correctly.
media_markets = Table().read_table("media_markets.csv")
media_markets.show(5)

Now, let's do the geographic analysis. Find the voting results for each county. You can usually find this by looking at each state's Secretary of State website and searching for election returns. Create a new Excel sheet with a table that has 5 columns: 

- `County` (the county that has the data), 
- `Winner` (the party that won the county). **Only use the label "Democrat" or "Republican"** 
- `# Voters D` (the # of voters for the Democratic candidate), 
- `# Voters R` (the # of voters for the Republican candidate), and
- `Election` (the election you are analyzing, such as the 2016 Presidential or the 2018 Midterm). 

The following table is an example for Pennsylvania with 1 entry: 

| County | Winner | # Voters D | # Voters R | Election |
| ---------| ------ | ------ | ---------- | ---------- | 
| Allegheny | Democrat | 367617 | 259480 | 2016 Presidential | 


Save this file in Excel **as a csv**, with the file name as **"elections.csv"** and upload it to the directory of this notebook. You can do so by clicking on the toolbar the top, then File > Open... This will open a new tab. In that tab, click on Upload on the top right and upload your file to the directory.


In [ ]:
## This cell should show a table with your data if you did this correctly.
elections = Table().read_table("elections.csv")
elections.show(5)

In [ ]:
## Now that we have all of the data, let's combine our datasets. Run this cell.
geo_voters = media_markets.join("County", elections, "County")
geo_voters.show(5)

## Step 2: Data Consolidation

The following cells will use your data and consolidate them by Media Market. For this section, **just run all of the cells and replace code when prompted** and read the outputs!


In [ ]:
# How is your data broken down by Media Market?
# This code will summarize the data by media market. Run this cell. 
def status(tbl):
    grouped = tbl.group(["Media Market", "Election"], sum)
    dems = grouped.column("# Voters D sum")
    reps = grouped.column("# Voters R sum")
    winners = dems >= reps # Assuming no tie in votes exists in the data
    results = []
    for i in winners:
        if i == True:
            results.append("Democrat")
        else:
            results.append("Republican")
    final_table = grouped.select("Media Market", "# Voters D sum", "# Voters R sum", "Election").with_column("Winner", results)
    return final_table

In [ ]:
# Run this cell. 
# Getting your elections:
your_elections = geo_voters.group("Election").column(0)

# Doing analysis by each election, finding the winner for that election  
overall_results = Table().with_columns("Media Market", [],
                                      "# Voters D sum", [],
                                      "# Voters R sum", [],
                                       "Election", [],
                                      "Winner", [])
for election in your_elections:
    by_election = geo_voters.where("Election", election)
    overall_results.append(status(by_election))
    
overall_results.show(5) # Show overall totals by media market in each election - see top table; 
# change to .show() to see all rows
    
# Combine the results and find how that market voted for all 4 elections - see bottom table
mm_breakdown = overall_results.group("Media Market", list).select("Media Market", "Winner list")
mm_breakdown.show()

In [ ]:
# Now, let's add a label of "Saint", "Sinner", or "Salvageable" for each Media Market in each election
# We will summarize winner in each market as a quantitative score. A Republican win will be scored as -1,
# A Democratic win will be +1. Run this cell.

mm_results = []
for i in overall_results.column("Winner"):
    if i == "Republican":
        mm_results.append(-1)
    elif i == "Democrat":
        mm_results.append(1)
        
scored_overall = overall_results.with_column("Score", mm_results)
scored_overall.show(5)

In [ ]:
# Combine the data so we can get a score for each Media Market. 
# If a Media Market has a score of -4, it is a "Saint" for the Republican Party;
# a score of +4, it is a "Saint" for the Democratic Party.
# Otherwise, it is a salvageable Media Market. Run this cell.

score_by_mm = scored_overall.group("Media Market", sum).select("Media Market", "Score sum")
score_by_mm.show(5)

In [ ]:
# Finally, add the label of Saint/Sinner/Salvageable to each Media Market. 
# COMPLETE THIS CODE:
# If you are running a Republican candidate, replace SCORE_TBL with score_repub.
# If you are running a Democratic candidate, replace SCORE_TBL with score_dems.
score_repub = Table().with_columns("Status", ["Saint", "Sinner"] + ["Salvageable"] * 7,
                             "Score", [-4, 4, -3, -2, -1, 0, 1, 2, 3])

score_dems = Table().with_columns("Status", ["Saint", "Sinner"] + ["Salvageable"] * 7,
                             "Score", [4, -4, -3, -2, -1, 0, 1, 2, 3])

score_labeled = score_by_mm.join("Score sum", SCORE_TBL, "Score") # make sure to replace SCORE_TBL!
score_labeled.show() # See all media markets and status

In [ ]:
# For your interest: what MMs are your "Saint"s, "Sinner"s, or "Salvageable"s?
# REPLACE THIS CODE: Replace "Saint" in the code with the group you want to check.
score_labeled.where("Status", "Saint").column("Media Market")

## Step 3: Data Visualization

Lastly, let's do some data visualization so we can compare data across media markets and by election. Like the last section, just run the cells and replace code when necessary.

In [ ]:
# First, let's just visualize the scores (i.e. who won the elections?)
# Scores to the right say this MM voted for Democrats
# Scores to the left say this MM voted for Republicans
# No bar means this group is split 50/50 
# Run this cell. 
score_labeled.barh("Media Market", "Score sum")

In [ ]:
# So, by how much did each party win each Media Market in each election?
# Let's go create some proportions to standardize the data. Run this cell. 
total_votes = overall_results.column("# Voters D sum") + overall_results.column("# Voters R sum")
with_margin = overall_results.with_columns("Prop Voters D", overall_results.column("# Voters D sum") / total_votes,
                                           "Prop Voters R", overall_results.column("# Voters R sum") / total_votes)

def make_graph(election):
    with_margin.where("Election", election).barh("Media Market", ["Prop Voters D", "Prop Voters R"])
    
with_margin.where("Election", "2016 Presidential").show() 
# Replace "2016 Presidential" with the election you want to view; keep the name in quotes

In [ ]:
# Let's see the breakdown of votes per election. Run this cell. You'll get a lot of graphs!
for i in your_elections:
    make_graph(i)
    plots.title(i)

# And that's it! 

That is your geographic analysis. As stated in the prompt, try doing your demographic analysis in an Excel sheet (it's a lot less data, so it will be much less work than this amount of code). 

Remember to write up an essay discussing these findings along with your demographic analysis.

If you have any questions about the analysis or need any help, again, please email me at castro.ian@berkeley.edu. 

Otherwise, if you found this interesting or useful, let me know! I work for the Data Science Education Program and if you would be interested in a political data science course or something similar (such as including more data-enabled analyses in your other courses), please fill out the survey below. Your comments and feedback will be very useful and greatly appreciated. Otherwise, if you're interested in working in data science education, such as curriculum and course development, consider applying to the Division of Computing, Data Science, and Society. 

## Feedback Survey: https://tinyurl.com/106voterdata

Thanks! -- Ian